In [2]:
import pandas as pd
import pickle
import numpy as np

# 데이터 불러오기
final_df = pd.read_csv("/content/drive/MyDrive/dataset/final(0519).csv")
# final_df_price = pd.read_csv("/content/drive/MyDrive/dataset/final_price(0502).csv")

In [3]:
# 결측치 처리
final_df["Title"].fillna("제목 미입력", inplace=True)
final_df["LocationDescription"].fillna("위치 정보 없음", inplace=True)
final_df["MainDescription"].fillna("상세 설명 없음", inplace=True)
final_df["SpaceDescription"].fillna("공간 설명 없음", inplace=True)
final_df["GuestAccessDescription"].fillna("이용 가능 공간 정보 없음", inplace=True)
final_df["Host_ResponseRate"].fillna("응답률 정보 없음", inplace=True)
final_df["Host_ResponseTime"].fillna("응답 시간 정보 없음", inplace=True)
final_df["amenities_SelfCheckIn"].fillna("셀프 체크인 여부 모름", inplace=True)
final_df["CheckIn_Start"].fillna("체크인 시간 모름", inplace=True)
final_df["CheckOut_End"].fillna("체크아웃 시간 모름", inplace=True)

<ipython-input-3-1f8fbedf4aae>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df["Title"].fillna("제목 미입력", inplace=True)
<ipython-input-3-1f8fbedf4aae>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

In [4]:
# 매핑 딕셔너리 정의
neighborhood_mapping = {
    "Ma-po-gu": "마포구",
    "Yong-san-gu": "용산구",
    "Yongsan-gu": "용산구",
    "Gang-nam-gu": "강남구",
    "Jong-no-gu": "종로구",
    "Jung-gu": "중구",
    "Myeong-dong": "명동",
    "Gwa-nak-gu": "관악구",
    "Yeong-deung-po-gu": "영등포구",
    "Gwang-jin-gu": "광진구",
    "Seo-cho-gu": "서초구",
    "Seocho-gu": "서초구",
    "Dong-dae-mun-gu": "동대문구",
    "Song-pa-gu": "송파구",
    "Seo-dae-mun-gu": "서대문구",
    "Seong-buk-gu": "성북구",
    "Eun-pyeong-gu": "은평구",
    "Seong-dong-gu": "성동구",
    "Dong-jak-gu": "동작구",
    "Gang-buk-gu": "강북구",
    "Sin-chon-dong": "신촌동",
    "Jung-nang-gu": "중랑구",
    "Seong-su 2 ga 1 dong": "성수2가1동",
    "Gu-ro-gu": "구로구",
    "Geum-cheon-gu": "금천구",
    "No-won-gu": "노원구",
    "Do-bong-gu": "도봉구",
    "Goyang-si": "고양시"
}

def map_neighborhood_to_korean(name: str) -> str:
    return neighborhood_mapping.get(name, name)  # 없는 값은 그대로 반환

# 새 컬럼 생성
final_df["LocalizedNeighbourhood_ML"] = final_df["LocalizedNeighbourhood_ML"].apply(map_neighborhood_to_korean)

In [8]:
final_df1 = final_df.iloc[150:300,:]
final_df2 = final_df.iloc[5000:,:]

In [6]:
final_df.columns

Index(['Airbnb_ListingID', 'Title', 'Location', 'Lat', 'Lng',
       'LocalizedNeighbourhood_ML', 'LocationDescription', 'MainDescription',
       'SpaceDescription', 'GuestAccessDescription', 'Bathrooms', 'Bedrooms',
       'Beds', 'PersonCapacity', 'Amenities', 'RoomType_Clean',
       'SpaceType_Clean', 'isSuperhost', 'Host_ResponseRate',
       'Host_ResponseTime', 'HostLanguages_Verbose', 'amenities_SelfCheckIn',
       'CheckIn_Start', 'CheckOut_End', 'ReviewComments', 'Rating',
       'ReviewCounts', 'Review_Length', 'ReviewSummary', 'title_korean'],
      dtype='object')

In [ ]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import lru_cache

# OpenAI API 키 설정
client = OpenAI(api_key="")

# 중복 요청 방지 캐시
@lru_cache(maxsize=50000)
def call_openai_cached(text):
    prompt = f"""입력 문장이 한국어라면 그대로 출력하고, 영어라면 한국어로 번역하세요.:
"{text}" """
    for attempt in range(10):  # 최대 3회 재시도
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "당신은 정확하고 깔끔한 번역가입니다."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
                timeout=10  # 초당 타임아웃 설정
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if attempt == 2:
                print(f"🔴 Failed after 3 tries: {text[:50]}... | Error: {e}")
                return text
            time.sleep(1 + attempt)  # 점진적 백오프

# 번역 대상: Title 열
texts = final_df1['Amenities'].tolist()

# 병렬 번역 함수
def parallel_translate(texts, max_workers=10):
    results = [None] * len(texts)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(call_openai_cached, text): i for i, text in enumerate(texts)}

        for future in tqdm(as_completed(futures), total=len(texts)):
            i = futures[future]
            try:
                results[i] = future.result()
            except Exception as e:
                print(f"🔴 Unexpected error: {e}")
                results[i] = texts[i]
    return results

# 실행
translated_titles = parallel_translate(texts)
final_df1["Amenities_korean"] = translated_titles

100%|██████████| 150/150 [01:07<00:00,  2.23it/s]
<ipython-input-9-a4ed73595afb>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df1["Amenities_korean"] = translated_titles


In [ ]:
final_df = pd.concat([final_df1, final_df2])

In [ ]:
final_df.to_csv('/content/drive/MyDrive/dataset/final(0519).csv', index=False)
# final_df["amenities_korean"] = final_df["Amenities"].progress_apply(translate_to_korean)

In [ ]:
# ✅ 1. CSV -> Pickle 변환 + FAISS 준비용 벡터 저장 스크립트
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 임베딩 텍스트 구성 함수
def build_embedding_text(row):
    if row['PersonCapacity'] >= 6:
        capacity_tag = "단체 여행객에게 적합한 넉넉한 공간입니다.\n"
    elif row['PersonCapacity'] >= 3:
        capacity_tag = "작은 그룹이나 가족 여행에 적당한 숙소입니다.\n"
    else:
        capacity_tag = "1~2인 여행자에게 알맞은 공간입니다.\n"

    return (
        f"리스팅ID: {row['Airbnb_ListingID']}\n"
        f"숙소 이름: {row['title_korean']}\n"
        f"위치: {row['LocalizedNeighbourhood_ML']}\n"
        f"위치 설명: {row['LocationDescription']}\n"
        f"숙소 소개: {row['MainDescription']}\n"
        f"공간 설명: {row['SpaceDescription']}\n"
        f"최대 수용 인원: {row['PersonCapacity']}명\n"
        f"{capacity_tag}"
        f"욕실 수: {row['Bathrooms']} / 침실 수: {row['Bedrooms']} / 침대 수: {row['Beds']}\n"
        f"슈퍼호스트 여부: {'예' if row['isSuperhost'] else '아니오'}\n"
        f"호스트 사용 언어: {row['HostLanguages_Verbose']}\n"
        f"별점: {row['Rating']}점 ({row['ReviewCounts']}개 리뷰)\n"
        f"어메니티: {row['Amenities']}\n"
        f"리뷰 요약: {row['ReviewSummary']}\n"
    )


# 임베딩 텍스트와 벡터 생성
final_df['embedding_text'] = final_df.apply(build_embedding_text, axis=1)
embeddings = embedding_model.encode(final_df['embedding_text'].tolist(), show_progress_bar=True, batch_size=64)

# numpy array로 변환
dense_vectors = np.array(embeddings)

# 저장 (Listing ID 포함 DF 유지)
with open("/content/drive/MyDrive/dataset/airbnb_data_준회.pkl", "wb") as f:
    pickle.dump({
        "df": final_df[['Airbnb_ListingID', 'embedding_text'] +
                      [c for c in final_df.columns if c not in ['Airbnb_ListingID', 'embedding_text']]],
        "vectors": dense_vectors
    }, f)

print("✅ Pickle 저장 완료: /content/drive/MyDrive/dataset/airbnb_data_준회.pkl")

Batches:   0%|          | 0/153 [00:00<?, ?it/s]

✅ Pickle 저장 완료: /content/drive/MyDrive/dataset/airbnb_data_준회.pkl


In [ ]:
print(final_df_price.isnull().sum().to_string())

Airbnb_ListingID           0
Calendar_Date              0
isAvailable                0
Min_Nights                 0
Max_Nights                 0
Available_For_Checkin      0
Available_For_Checkout     0
Guests                     0
Price_Per_Night            0
Cleaning_Fee               0
Airbnb_Service_Fee         0
Available_Ranges           0
Price_Per_Night_mean       0
Cleaning_Fee_mean          0
Airbnb_Service_Fee_mean    0
total_price_mean           0
total_price                0


In [ ]:
final_df_price.head()

,Airbnb_ListingID,Calendar_Date,isAvailable,Min_Nights,Max_Nights,Available_For_Checkin,Available_For_Checkout,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,Available_Ranges,Price_Per_Night_mean,Cleaning_Fee_mean,Airbnb_Service_Fee_mean,total_price_mean,total_price
0,1000084126553197219,2025-01-06,True,3,365,True,True,2.0,80240.0,23600.0,0.0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046.25,23600.0,0.0,122646.25,103840.0
1,1000084126553197219,2025-01-07,True,3,365,True,True,2.0,80240.0,23600.0,0.0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046.25,23600.0,0.0,122646.25,103840.0
2,1000084126553197219,2025-01-08,True,3,365,True,True,2.0,80240.0,23600.0,0.0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046.25,23600.0,0.0,122646.25,103840.0
3,1000084126553197219,2025-01-09,True,5,365,True,True,2.0,80240.0,23600.0,0.0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046.25,23600.0,0.0,122646.25,103840.0
4,1000084126553197219,2025-01-13,True,3,365,True,True,2.0,80240.0,23600.0,0.0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046.25,23600.0,0.0,122646.25,103840.0


In [ ]:
final_df_price['price_diff']= final_df_price['Price_Per_Night'] - final_df_price['Price_Per_Night_mean']
price_comment = (
    "평균보다 저렴한 편입니다." if price_diff < 0 else
    "평균보다 다소 높은 편입니다." if price_diff > 0 else
    "평균과 동일한 수준입니다."
)
f"요금 비교: {price_comment}\n"

,Available_Ranges
0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
1,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
2,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
3,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
4,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
...,...
1922840,"12월: 1-6, 8-13, 15-16, 16-17, 17-18, 18-19, 19..."
1922841,"12월: 1-6, 8-13, 15-16, 16-17, 17-18, 18-19, 19..."
1922842,"12월: 1-6, 8-13, 15-16, 16-17, 17-18, 18-19, 19..."
1922843,"12월: 1-6, 8-13, 15-16, 16-17, 17-18, 18-19, 19..."


In [ ]:
# ✅ 1. CSV -> Pickle 변환 + FAISS 준비용 벡터 저장 스크립트
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 임베딩 텍스트 구성 함수
def build_embedding_text(row):
    # 가격 차이 및 변화율 계산
    price_diff = row['total_price'] - row['total_price_mean']
    price_change_ratio = (price_diff / row['total_price_mean']) * 100 if row['total_price_mean'] else 0

    # 요금 비교 문구 ('평소' 기준 ±5% 허용)
    if abs(price_change_ratio) <= 5:
        price_comment = "평소 수준의 요금입니다."
    elif price_change_ratio < -5:
        price_comment = "평소보다 저렴한 편입니다."
    else:
        price_comment = "평소보다 다소 높은 편입니다."

    # 특가 여부 태그
    special_offer_note = " 특가 숙소로 분류됩니다.\n" if price_change_ratio < -20 else ""

    return (
        f"리스팅ID: {row['Airbnb_ListingID']}\n"
        f"예약 날짜: {row['Calendar_Date']}\n"
        f"예약 가능 여부: {'가능' if row['isAvailable'] else '불가능'}\n"
        f"예약 가능 날짜: {row['Available_Ranges']}\n"
        f"최소 숙박일: {row['Min_Nights']}박 / 최대 숙박일: {row['Max_Nights']}박\n"
        f"체크인 가능 여부: {'가능' if row['Available_For_Checkin'] else '불가능'} / "
        f"체크아웃 가능 여부: {'가능' if row['Available_For_Checkout'] else '불가능'}\n"
        f"예약 인원: {row['Guests']}명\n"
        f"1박 요금: {row['Price_Per_Night']}원\n"
        f"청소비: {row['Cleaning_Fee']}원 / 에어비앤비 수수료: {row['Airbnb_Service_Fee']}원\n"
        f"1박 총요금: {row['total_price']}원\n"
        f"요금 비교: {price_comment} (평소 대비 {price_change_ratio:.1f}% {'상승' if price_change_ratio > 0 else '하락' if price_change_ratio < 0 else '변동 없음'})\n"
        f"{special_offer_note}"
        f"평균 1박 요금: {row['Price_Per_Night_mean']}원\n"
        f"평균 청소비: {row['Cleaning_Fee_mean']}원 / 평균 에어비앤비 수수료: {row['Airbnb_Service_Fee_mean']}원\n"
        f"평균 1박 총요금: {row['total_price_mean']}원\n"
    )



# 임베딩 텍스트와 벡터 생성
final_df_price['embedding_text'] = final_df_price.apply(build_embedding_text, axis=1)
embeddings = embedding_model.encode(final_df_price['embedding_text'].tolist(), show_progress_bar=True, batch_size=64)

# numpy array로 변환
dense_vectors = np.array(embeddings)

# 저장 (Listing ID 포함 DF 유지)
with open("/content/drive/MyDrive/dataset/airbnb_data_price(전처리추가).pkl", "wb") as f:
    pickle.dump({
        "df": final_df_price[['Airbnb_ListingID', 'embedding_text'] +
                      [c for c in final_df_price.columns if c not in ['Airbnb_ListingID', 'embedding_text']]],
        "vectors": dense_vectors
    }, f)

print("✅ Pickle 저장 완료: /content/drive/MyDrive/dataset/airbnb_data_price(전처리추가).pkl")

Batches:   0%|          | 0/30045 [00:00<?, ?it/s]

KeyboardInterrupt: 